<a href="https://colab.research.google.com/github/ericsdata/colinsbeer/blob/main/src/BeerSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd

dat = pd.read_csv(r'txt_train.csv')
dat

,review_text,good_score
0,"UPDATED: MAR 6, 2004 Amber, with a wonderful o...",0.0
1,"I did not really like this beer. Grainy, bitt...",0.0
2,I was quite pleased with this beer. I tried a ...,0.0
3,Tried UK 3% version in 440ml can. Light in co...,0.0
4,Clean on the palate with a musty biscuity malt...,0.0
...,...,...
6995,Light Yellow Coloring. Similiar to the Last.....,0.0
6996,"UPDATED: FEB 3, 2004 Draught at The Rover, Got...",0.0
6997,"Syrupy, molasses nose; caramel-Coke color; foa...",1.0
6998,"Einbecker ""Schwarzbier"" is a ruby brown lager ...",0.0


In [2]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 34.7 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
     |████████████████████████████████| 895 kB 57.9 MB/s 
     |████████████████████████████████| 3.3 MB 58.9 MB/s 
     |████████████████████████████████| 596 kB 68.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import tensorflow_datasets as tfds
import tensorflow as tf
import torch

from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification


In [4]:
import numpy as np
import random
import math
#from random import sample

random.seed(1144)

population = [i for i in dat.index]


train_proportion = .7
train_size = math.floor(len(population) * train_proportion)

train_idx = random.sample(population, train_size)

dat.loc[train_idx]

mldat = {"train" : dat.loc[train_idx].to_dict(orient = "records"),
        "test" : dat.loc[~dat.index.isin(train_idx)].to_dict(orient = "records")}




In [ ]:


from sklearn.model_selection import train_test_split

reviews = dat['review_text'].to_list()




full_labs = dat['good_score'].to_list()

X_train, X_test, y_train, y_test = train_test_split(reviews, full_labs, test_size=0.33, random_state=42)

Continually **keep** trying
https://www.datatrigger.org/post/nlp_hugging_face/

In [5]:
train =  dat.loc[train_idx]
test = dat.loc[~dat.index.isin(train_idx)]

In [13]:
class Beer_Data(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

In [23]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)


### Make encodings of the data

train_encodings = tokenizer(train['review_text'].astype(str).to_list(), truncation = True, padding = True)
test_encodings = tokenizer(test['review_text'].astype(str).to_list(), truncation = True, padding = True)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.15.0",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytor

In [24]:
train_dataset = Beer_Data(train_encodings, train['good_score'].to_list())
test_dataset = Beer_Data(test_encodings, test['good_score'].to_list())

In [18]:
train_dataset.__getitem__(1)

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0

In [25]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

labels = [0,1]

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
)


trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=test_dataset             
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [26]:
trainer.train()
trainer.save_model("beer_review_model")

***** Running training *****
  Num examples = 4900
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1839


Step,Training Loss
500,0.033400
1000,0.007100
1500,0.001300


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to beer_review_model
Configuration saved in beer_review_model/config.json
Model weights saved in beer_review_model/pytorch_model.bin


In [27]:
val = pd.read_csv(r'txt_test.csv')

val_encodings = tokenizer(val['review_text'].astype(str).to_list(), truncation = True, padding = True)

val_dataset = Beer_Data(val_encodings, val['good_score'].to_list())

predictions = trainer.predict(val_dataset)

***** Running Prediction *****
  Num examples = 3000
  Batch size = 4


In [35]:
test_results=val.copy(deep=True)
test_results["label_int_pred_transfer_learning"]=predictions.label_ids
test_results
#test_results['label_pred_transfer_learning']=test_results['label_int_pred_transfer_learning'].apply(lambda x:labels[x])

#test_results[test_results["good_score"]!=test_results["label_pred_transfer_learning"]].head()

,review_text,good_score,label_int_pred_transfer_learning
0,"Well, tipically bavaria, cheap and fast, not m...",0.0,0.0
1,Packaging sure made this one look deceptively ...,0.0,0.0
2,"Bubbly white head, golden see through. Malty,...",1.0,1.0
3,Bottle pours a light yellow with a thin white ...,0.0,0.0
4,Bottle. Dark ruby colour with a tan head. Arom...,0.0,0.0
...,...,...,...
2995,750ml bottle. Poured with a huge rocky white f...,0.0,0.0
2996,Growler at Shenadoah Throwdown thanks to wicke...,0.0,0.0
2997,"This beer could be a hit with a ""whip out your...",1.0,1.0
2998,Caramel and bitter woody scent. Deep ruby colo...,0.0,0.0


In [37]:
pd.crosstab(test_results['good_score'], test_results['label_int_pred_transfer_learning'])

label_int_pred_transfer_learning,0.0,1.0
good_score,,
0.0,2891,0
1.0,0,109


In [ ]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)

In [ ]:
max_len = 512
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

test_seq = mldat['train'][1]['review_text']
test_seq2 = mldat['train'][2]['review_text']

seqs = [mldat['train'][1]['review_text'],mldat['train'][2]['review_text'] ]
tokens = tokenizer(seqs, padding = True, 
                            truncation = True, return_tensors = 'pt')
print(tokens)


ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])

model(**tokens)

{'input_ids': tensor([[  101,   101, 26071,  1012, 10364,  2015,  1037,  5024,  2422,  3756,
          2007,  4317,  2132,  1998,  2152,  6351,  3370,  1012, 14747,  2066,
          2087,  7935, 18007,  1010,  2021,  2200,  8217,  1012,  2023,  5404,
          2018,  1037,  2844,  3988, 20943,  5510,  1010,  2029,  2059, 23946,
          2000,  1037,  2300,  2044, 10230,  2618,  1012,  2023,  2419,  2000,
          1037,  4326,  5688,  2004,  2009,  2001,  2667,  2000,  2022,  2119,
          1037,  5744,  4392,  3085,  5404,  1998,  2242,  2007,  6805,  1012,
          1996,  5404,  2001,  2036, 19638,  2084,  3517,  1010,  1998,  1996,
          2132,  8314,  2000, 19219,  2015,  1012,  3452,  1037,  4392,  3085,
          5404,  1010,  2074,  2025,  2200, 22249,  1012,   102,   102],
        [  101,   101, 16958,  2200, 23353,  2100,  1998,  2025,  2428,  2172,
         28126,  1010,  1045,  2876,  2102,  2175,  2041,  1997,  2026,  2126,
          2000,  2031,  2023,  2153,   102, 

SequenceClassifierOutput([('logits', tensor([[ 3.0460, -2.5669],
                                   [ 3.2350, -2.7345]], grad_fn=<AddmmBackward0>))])

In [ ]:
sents = [d['review_text'] for d in mldat["train"]]
tokenized_reviews = tokenizer(sents, padding = True, 
                            truncation = True, return_tensors = 'pt')



len(tokenized_reviews['input_ids'])

4900

In [ ]:
train_InputExamples

4181    InputExample(guid=None, text_a='[CLS]  [SEP]',...
3943    InputExample(guid=None, text_a='[CLS] Bottled....
4025    InputExample(guid=None, text_a='[CLS] Tastes v...
3991    InputExample(guid=None, text_a='[CLS] Bottle: ...
1436    InputExample(guid=None, text_a='[CLS] Pours cl...
                              ...                        
2846    InputExample(guid=None, text_a='[CLS] Bottle p...
6292    InputExample(guid=None, text_a='[CLS] Can. Pou...
1719    InputExample(guid=None, text_a='[CLS] UPDATED:...
1571    InputExample(guid=None, text_a='[CLS] Bottle, ...
1208    InputExample(guid=None, text_a='[CLS] UPDATED:...
Length: 4900, dtype: object

In [ ]:
from transformers import InputExample, InputFeatures

train =  dat.loc[train_idx]
test = dat.loc[~dat.index.isin(train_idx)]

def convert_data_to_examples(train, test, review, sentiment): 
    train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[review], 
                                                          label = x[sentiment]), axis = 1)

    validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[review], 
                                                          label = x[sentiment]), axis = 1,)
  
    return train_InputExamples, validation_InputExamples

train_InputExamples, validation_InputExamples = convert_data_to_examples(train,  test, 'review_text',  'good_score')


In [ ]:



 ## Import BERT Model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

## Moddel inputs holds two arrays
    ## input_ids: list of id of word vector
    ## attention_mask
    
n_batch = 10

batch = sequence [:n_batch]
batch_labs = labels [:n_batch]

model_inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
#model_inputs = tokenizer(sequence, padding = True, truncation = True, return_tensors= 'pt')


## List of IDs
ids = tokenizer.convert_tokens_to_ids(model_inputs)


#batch = tokenizer(few_reviews, padding="max_length", truncation=True, return_tensors="pt")

## !!! Attaching class to model inputs
model_inputs['labels'] = torch.tensor(batch_labs)

'''
### Choosing tokenizer
####    A) Keep reviews by uid
####    B) Sentence strings associated with a particular style

output = model(**model_inputs)

i = 0

for out in range(0,len(output)):

    print(output[i])
    i+=1
'''

'\n### Choosing tokenizer\n####    A) Keep reviews by uid\n####    B) Sentence strings associated with a particular style\n\noutput = model(**model_inputs)\n\ni = 0\n\nfor out in range(0,len(output)):\n\n    print(output[i])\n    i+=1\n'

In [ ]:

optimizer = AdamW(model.parameters())
loss = model(**model_inputs).loss
loss.backward()
optimizer.step()